In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

import os

c:\Users\HOME\anaconda3\envs\conda_env_genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["GOOGLE_API_KEY"]="api-key"

gemini_llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash", max_output_tokens=350, temperature=0)

In [3]:
loader=PyPDFLoader("/ml.pdf")

docs=loader.load()

In [4]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
data_splits=splitter.split_documents(docs)

In [5]:
g_embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

db=Chroma.from_documents(documents=data_splits, embedding=g_embeddings)

db

In [6]:
# g_embeddings.embed_query("What is Machine Learning?") #Returns vector for this question

In [7]:
# Designing the ChatPrompt Template
chat_prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

combined_chain=create_stuff_documents_chain(gemini_llm, chat_prompt)

combined_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \nI will tip you $1000 if the user finds the answer helpful. \n<context>\n{context}\n</context>\nQuestion: {input}'))])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', temperature=0.0, max_output_tokens=350, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000268542CAC30>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x00000268659F4DA0>, default_metadata=())
| StrOutputParser(), config={'run_na

In [9]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000026865E5EE10>)

In [10]:
from langchain.chains import create_retrieval_chain

retriver_chain=create_retrieval_chain(retriever,combined_chain)

retriver_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000026865E5EE10>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \nI will tip you $1000 if the user finds the answer helpful. \n<context>\n{context}\n</context>\nQuestion: {input}'))])
            | ChatGoogleGener

In [12]:
response_from_RAG=retriver_chain.invoke({"input":"What is Supervised Learning?"})

# print(response_from_RAG)

response_from_RAG['answer']

'Here\'s how we can break down the answer to the question "What is Supervised Learning?" based on the provided context:\n\n1. **Identify the Key Paragraph:** The context provides a definition of supervised learning in the first paragraph. \n2. **Extract the Definition:** The paragraph states that supervised learning is a type of machine learning where we aim to predict an outcome from a given input. We achieve this by using examples of input/output pairs to build a model.\n3. **Summarize the Key Points:**  Supervised learning involves:\n    * **Predicting outcomes:**  The goal is to predict what will happen based on given input.\n    * **Training data:**  We use examples of input/output pairs (called a training set) to build the model.\n    * **Generalization:**  The model should be able to make accurate predictions on new, unseen data that has similar characteristics to the training set.\n\n**Therefore, Supervised Learning is a type of machine learning where we train a model on labele